In [1]:
import json
from collections import defaultdict
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("../Data/Interim/some_cmd_split.csv", sep=",")
df = df.loc[:,["full_text", "text1", "text2", "text3", "subset"]]
print(df.shape)
# добавляем кома
df_onecmd = pd.read_csv("../Data/Interim/12052022_one_cmd_split.csv", sep=",")
df_onecmd = df_onecmd.sample(5000, random_state=42)
df_onecmd = df_onecmd.loc[df_onecmd["subset"].isin(["train", "valid"]), ["x", "subset"]]
df_onecmd["full_text"] = df_onecmd["x"]
df_onecmd["text1"] = df_onecmd["x"]
df_onecmd["text2"] = np.empty_like(df_onecmd["x"], dtype=np.float32)
df_onecmd["text3"] = np.empty_like(df_onecmd["x"], dtype=np.float32)

df = pd.concat([df, df_onecmd])

(16722, 5)


/tmp/ipykernel_22266/1976607233.py:1: DtypeWarning: Columns (3,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../Data/Interim/some_cmd_split.csv", sep=",")


In [3]:
nerInput_train = {
    "sentence_id": [],
    "words": [],
    "labels": []
}
nerInput_valid = {
    "sentence_id": [],
    "words": [],
    "labels": []
}
nerInput = None
for ir, row in df.iterrows():
    if row["subset"]=="train":
        nerInput = nerInput_train
    if row["subset"]=="valid":
        nerInput = nerInput_valid

    start_1 = row["full_text"].find(row["text1"])
    assert start_1 >= 0
    start_2 = None
    if type(row['text2']) != float:
        start_2 = row["full_text"].find(row["text2"], start_1 + len(row["text1"]))    
        assert start_2 >= start_1
    start_3 = None
    if type(row['text3']) != float:
        start_3 = row["full_text"].find(row["text3"], start_2 + len(row["text2"]))
        assert start_3 >= start_2
    lastWordPos = 0
    for word in row["full_text"].split():
        nerInput["sentence_id"].append(ir)
        nerInput["words"].append(word)
        wordPos = row["full_text"].find(word, lastWordPos)
        assert wordPos!=-1
        lastWordPos = wordPos + len(word)
        if wordPos >= start_1 and wordPos < (start_1+len(row["text1"])):
            nerInput["labels"].append("CMD1")
        elif (start_2 is not None) and (wordPos >= start_2) and (wordPos < (start_2+len(row["text2"]))):
            nerInput["labels"].append("CMD2")
        elif (start_3 is not None) and (wordPos >= start_3) and (wordPos < (start_3+len(row["text3"]))):
            nerInput["labels"].append("CMD3")
        else:
            nerInput["labels"].append("O")
train_df = pd.DataFrame(nerInput_train)
valid_df = pd.DataFrame(nerInput_valid)

In [5]:
ir = 5000
print(df.loc[ir, "full_text"])
ner_commands= [[],[],[]]
for ir_2, row in train_df[train_df["sentence_id"]==ir].iterrows():
    if row["labels"]=="CMD1":
        ner_commands[0].append(row["words"])
    if row["labels"]=="CMD2":
        ner_commands[1].append(row["words"])
    if row["labels"]=="CMD3":
        ner_commands[2].append(row["words"])
print(df.loc[ir, "text1"], "|", " ".join(ner_commands[0]))
print(df.loc[ir, "text2"], "|", " ".join(ner_commands[1]))
print(df.loc[ir, "text3"], "|", " ".join(ner_commands[2]))

обойди окрестности по кругу размером 17 метров после всего тяни на южный край на 4 м а там за лесом располагается камушек который нужно обскакивать
обойди окрестности по кругу размером 17 метров | обойди окрестности по кругу размером 17 метров
тяни на южный край на 4 м | тяни на южный край на 4 м
за лесом располагается камушек который нужно обскакивать | за лесом располагается камушек который нужно обскакивать


In [6]:
from simpletransformers.ner import NERModel, NERArgs
model_args = NERArgs()
model_args.evaluate_during_training = True
model_args.num_train_epochs = 10
model_args.output_dir = "/s/ls4/users/grartem/RL_robots/CommandClassifier/models/Preprocess_for_splitter_ntbk"

model = NERModel(
    "bert", 
    "/s/ls4/users/grartem/HuggingFace/models/rubert-tiny2",
    #"/s/ls4/users/grartem/RL_robots/CommandClassifier/models/Preprocess_for_splitter_ntbk/checkpoint-18500-epoch-10/",
    args=model_args,
    labels=["O", "CMD1", "CMD2", "CMD3"] # добавить тэг O, но по границам
)

Some weights of the model checkpoint at /s/ls4/users/grartem/HuggingFace/models/rubert-tiny2 were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassific

In [7]:
model_args.output_dir

'/s/ls4/users/grartem/RL_robots/CommandClassifier/models/Preprocess_for_splitter_ntbk'

In [8]:
# Train the model
model.train_model(train_df, eval_data=valid_df)

  0%|          | 0/29 [00:00<?, ?it/s]

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/1960 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CMD1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CMD2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CMD3 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


  0%|          | 0/6 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/373 [00:00<?, ?it/s]

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CMD1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CMD2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CMD3 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


(19600,
 defaultdict(list,
             {'global_step': [1960,
               2000,
               3920,
               4000,
               5880,
               6000,
               7840,
               8000,
               9800,
               10000,
               11760,
               12000,
               13720,
               14000,
               15680,
               16000,
               17640,
               18000,
               19600],
              'train_loss': [0.0024012785870581865,
               0.0036378283984959126,
               0.01069735735654831,
               0.010477759875357151,
               0.0010051507269963622,
               0.018477605655789375,
               0.00012169281399110332,
               0.0014449754962697625,
               6.0819445934612304e-05,
               0.00012475803669076413,
               0.00010830422979779541,
               0.00010404713248135522,
               2.8362641387502663e-05,
               4.189837272861041e-05,


In [9]:
# Evaluate the model
result, model_outputs, preds_list = model.eval_model(valid_df)

  0%|          | 0/6 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/373 [00:00<?, ?it/s]

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CMD1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CMD2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CMD3 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [10]:
predictions, raw_outputs = model.predict(df[df["subset"]=="valid"]["full_text"])

  0%|          | 0/7 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/389 [00:00<?, ?it/s]

# оценка

In [12]:
from sklearn.metrics import classification_report
flat_token_predictions = [list(x.values())[0] for sent in predictions for x in sent]
flat_token_true = valid_df["labels"].values
print(classification_report(flat_token_true, flat_token_predictions))

              precision    recall  f1-score   support

        CMD1       1.00      1.00      1.00     15260
        CMD2       1.00      1.00      1.00     10895
        CMD3       1.00      0.98      0.99      3127
           O       1.00      1.00      1.00      3154

    accuracy                           1.00     32436
   macro avg       1.00      0.99      1.00     32436
weighted avg       1.00      1.00      1.00     32436



In [25]:
i, errors = 0, 0
for ir, row in df[df["subset"]=="valid"].iterrows():
    text1, text2, text3 = [], [], []
    for token in predictions[i]:
        token, cls = list(token.items())[0]
        if cls=="CMD1":
            text1.append(token)
        if cls=="CMD2":
            text2.append(token)
        if cls=="CMD3":
            text3.append(token)
    text1 = " ".join(text1)
    text2 = " ".join(text2)
    text3 = " ".join(text3)
    
    try:
        assert text1 == row["text1"].strip()
        assert (text2=="" and type(row["text2"]) is float) or text2 == row["text2"].strip()
        assert (text3=="" and type(row["text3"]) is float) or text3 == row["text3"].strip()
        #assert (text3=="" and np.isnan(row["text3"])) or text3 == row["text3"].strip()
    except AssertionError:
        print("full_text:", row["full_text"])
        print("text1:", row["text1"], "predict:", text1)
        print("text2:", row["text2"], "predict:", text2)
        print("text3:", row["text3"], "predict:", text3)
        print()
        errors += 1
    i+=1
print("Errors", errors)

full_text: обскочи загородку что рядом с останцем который располагается около рытвины припусти к ольхе
text1: обскочи загородку что рядом с останцем который располагается около рытвины predict: обскочи загородку что рядом с останцем который располагается около рытвины ольхе
text2: припусти к ольхе predict: припусти к
text3: nan predict: 

full_text: пытайся обскакивать чащу которая справа от машины что левее дуба соверши поворот 600 в сторону севера
text1: пытайся обскакивать чащу которая справа от машины что левее дуба predict: пытайся обскакивать чащу которая справа от машины что левее дуба севера
text2: соверши поворот 600 в сторону севера predict: соверши поворот 600 в сторону
text3: nan predict: 

full_text: убеги к мужчине слева  неподалеку от валуна располагается упавшее дерево которое нужно разглядывать
text1: убеги к мужчине слева  predict: убеги к мужчине
text2: неподалеку от валуна располагается упавшее дерево которое нужно разглядывать predict: слева неподалеку от валуна ра

# проверим, как она хорошо предсказывает фразы из 1 команды, если не видела их

In [26]:
df = pd.read_csv("../Data/Interim/12052022_one_cmd_split.csv", sep=",")
predictions, raw_outputs = model.predict(df["x"])

  0%|          | 0/29 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/2661 [00:00<?, ?it/s]

In [27]:
errors_count = 0
for phrase in predictions:
    classes = list(set([list(x.values())[0] for x in phrase]))
    if len(classes)==1 and classes[0]=="CMD1":
        continue
    else:
        errors_count+=1
errors_count

400